In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv("C:/Users/Dell/Desktop/Deep_Learning/Data/insurance_data.csv")
df

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


#### Split train and test set

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[["age", "affordibility"]], 
                                                    df["bought_insurance"], test_size= 0.25, 
                                                    random_state= 25)

#### Scaling for better result

In [6]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_test_scaled["age"] = X_test_scaled["age"]/ 100
X_train_scaled["age"] = X_train_scaled["age"] / 100

In [7]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape = (2,),
                       activation = "sigmoid",
                       kernel_initializer='ones',
                       bias_initializer='zeros')
])

model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

model.fit(X_train_scaled, y_train, epochs = 5000)

Epoch 1/5000


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5238 - loss: 0.6748
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5238 - loss: 0.6745
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5238 - loss: 0.6742
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5238 - loss: 0.6739
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5238 - loss: 0.6735
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5238 - loss: 0.6732
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5238 - loss: 0.6729
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.5238 - loss: 0.6726
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5238 - loss: 0.6723
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5238 - loss: 0.6720
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.5238 - loss: 0.6717
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5238 - loss

KeyboardInterrupt: 

In [8]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[0.6950769 ],
       [0.60231143],
       [0.36539528],
       [0.6355475 ],
       [0.70102006],
       [0.735234  ],
       [0.61573863]], dtype=float32)

In [9]:
coef, intercept = model.get_weights()
coef, intercept

(array([[1.4098535],
        [1.0799162]], dtype=float32),
 array([-0.91858447], dtype=float32))

In [10]:
y  = coef[0]* X_test_scaled["age"] + coef[1] * X_test_scaled["affordibility"] + intercept
y

2     0.823963
10    0.415105
21   -0.552023
11    0.556091
14    0.852160
9     1.021342
0     0.471500
dtype: float64

In [11]:
def sigmoid(x):
    return 1/(1+ np.exp(-x))

In [12]:
sigmoid(y)

2     0.695077
10    0.602311
21    0.365395
11    0.635548
14    0.701020
9     0.735234
0     0.615739
dtype: float64

In [13]:
print(zip(np.array(sigmoid(y)), model.predict(X_test_scaled)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [14]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [15]:
def grad_desc(x1, x2, y_true, epochs, loss_threshold):
    w1 = w2 = 1
    bias = 0
    learning_rate = 0.5
    n = len(x1)

    for i in range(epochs):
        weighted_sum = w1 * x1 + w2 * x2 + bias
        y_pred = sigmoid(weighted_sum) 
        loss = log_loss(y_true, y_pred)

        dw1 = (1/n)* np.dot(np.transpose(x1), (y_pred-y_true))
        dw2 = (1/n)* np.dot(np.transpose(x2), (y_pred-y_true))
        dbias = np.mean(y_pred-y_true)

        w1 = w1- learning_rate * dw1
        w2 = w2- learning_rate * dw2
        bias = bias - learning_rate * dbias

        print(f"Epoch:{i}, w1:{w1}, bias:{bias}, Loss: {loss}")

        if loss <= loss_threshold:
            break
    return  w1,  w2, bias 




In [16]:
grad_desc(X_train_scaled['age'],X_train_scaled['affordibility'],y_train, 10000, 0.31)

Epoch:0, w1:0.977756901275278, bias:-0.10043710129941415, Loss: 0.6747999380836459
Epoch:1, w1:0.960507699092837, bias:-0.18856414662993784, Loss: 0.6526733236935852
Epoch:2, w1:0.9477865445427416, bias:-0.2655697055389144, Loss: 0.6361670909413256
Epoch:3, w1:0.9390773678629402, bias:-0.3327436526018175, Loss: 0.6239712896738469
Epoch:4, w1:0.9338564590825825, bias:-0.3913776692933585, Loss: 0.6149814505853841
Epoch:5, w1:0.931623201468376, bias:-0.4426951494022258, Loss: 0.6083184844719988
Epoch:6, w1:0.9319189588780146, bias:-0.4878094702578811, Loss: 0.6033131289642611
Epoch:7, w1:0.9343362964823616, bias:-0.5277048861614209, Loss: 0.5994729308282516
Epoch:8, w1:0.9385213375117396, bias:-0.5632331740497403, Loss: 0.5964447851677502
Epoch:9, w1:0.9441717930771973, bias:-0.5951199969303458, Loss: 0.5939803755895849
Epoch:10, w1:0.9510326045540535, bias:-0.6239764608404114, Loss: 0.5919075721270781
Epoch:11, w1:0.9588905268232906, bias:-0.6503128168125958, Loss: 0.5901082917807494
Epo

(10.226531555844678, 1.9463415996940443, -5.399345111025493)

# Building Neural Network manually

In [46]:
class Neural_networks:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_threshold ):
        self.w1, self.w2, self.bias = self.grad_desc(X["age"], X["affordibility"], y , epochs, loss_threshold)

    def predict(self, X_test):
        weighted_sum = self.w1 * X_test["age"] + self.w2 * X_test["affordibility"] + self.bias 
        return sigmoid(weighted_sum)

    def grad_desc(self, x1, x2, y_true, epochs, loss_threshold):
        w1 = self.w1
        w2 = self.w2
        bias = self.bias
        learning_rate = 0.5
        n = len(x1)

        for i in range(epochs):
            weighted_sum = w1 * x1 + w2 * x2 + bias
            y_pred = sigmoid(weighted_sum) 
            loss = log_loss(y_true, y_pred)

            dw1 = (1/n)* np.dot(np.transpose(x1), (y_pred-y_true))
            dw2 = (1/n)* np.dot(np.transpose(x2), (y_pred-y_true))
            dbias = np.mean(y_pred-y_true)

            w1 = w1- learning_rate * dw1
            w2 = w2- learning_rate * dw2
            bias = bias - learning_rate * dbias

            if i % 50 == 0:
                print(f"Epoch:{i}, w1:{w1}, bias:{bias}, Loss: {loss}")
            

            if loss <= loss_threshold:
                break
        return  w1,  w2, bias 


    



In [47]:
myNN = Neural_networks()

myNN.fit(X_train_scaled, y_train, epochs= 1000, loss_threshold = 0.40)

Epoch:0, w1:0.977756901275278, bias:-0.10043710129941415, Loss: 0.6747999380836459
Epoch:50, w1:1.4259089137047314, bias:-1.194107797310679, Loss: 0.5545238413204947
Epoch:100, w1:2.0463379809323157, bias:-1.6140677253980895, Loss: 0.5297843346763951
Epoch:150, w1:2.633130649478673, bias:-1.9211040855000148, Loss: 0.5118660756724157
Epoch:200, w1:3.1757247944899203, bias:-2.1722779900397473, Loss: 0.49746914907431855
Epoch:250, w1:3.672960653649412, bias:-2.3907253591580013, Loss: 0.48562113717655125
Epoch:300, w1:4.127253825904126, bias:-2.586719969185999, Loss: 0.47579320647480056
Epoch:350, w1:4.54222568575647, bias:-2.7653258474982176, Loss: 0.4676001456235326
Epoch:400, w1:4.92170813689654, bias:-2.9293950325720797, Loss: 0.4607390165492049
Epoch:450, w1:5.269342066912108, bias:-3.080786978691324, Loss: 0.4549677597555708
Epoch:500, w1:5.588436022995237, bias:-3.2208774018220585, Loss: 0.45009215637226785
Epoch:550, w1:5.881936927725986, bias:-3.3507767548477827, Loss: 0.445955916

In [48]:
myNN.predict(X_test_scaled)

2     0.767197
10    0.260446
21    0.101592
11    0.432282
14    0.793602
9     0.906540
0     0.324052
dtype: float64